In [36]:
from wikibaseintegrator.models import Qualifiers, References, Reference
from wikibaseintegrator import WikibaseIntegrator
from wikibaseintegrator import wbi_login
from wikibaseintegrator import datatypes
from wikibaseintegrator.wbi_config import config as wbi_config
from os.path import join, isfile, exists, isdir, abspath, pardir
from os import listdir, getcwd
import pandas as pd
from wikibaseintegrator.datatypes import (URL, CommonsMedia, ExternalID, Form, GeoShape, GlobeCoordinate, Item, Lexeme, Math, MonolingualText, MusicalNotation, Property, Quantity, Sense, String, TabularData, Time)
from wikibaseintegrator.wbi_enums import ActionIfExists, WikibaseDatePrecision, WikibaseRank, WikibaseSnakType

In [4]:
# Default variables
WDUSER, WDPASS = "Admin", "90J8XXXaO4Sr9^^Z"
# WDUSER, WDPASS = "Mohammadzainabbas", "fHh!%shFa6^h"

In [13]:
parent_dir = abspath(join(getcwd(), pardir))
data_dir = join(parent_dir, "data")
team_data = join(data_dir, "BDMA_Team_v1.csv")
project_data = join(data_dir, "BDMA_Projects_v1.csv")

In [2]:
def setup_config():
    """
    Set up WBI config to use local docker installation
    """
    wbi_config['MEDIAWIKI_API_URL'] = 'http://139.144.66.193:8181/api.php'
    wbi_config['SPARQL_ENDPOINT_URL'] = '"http://139.144.66.193:8282/proxy/wdqs/bigdata/namespace/wdq/sparql"'
    wbi_config['WIKIBASE_URL'] = 'http://wikibase.svc'

In [3]:
setup_config()

In [5]:
login = wbi_login.Login(user=WDUSER, password=WDPASS)

In [6]:
wbi = WikibaseIntegrator(login=login)

In [16]:
df = pd.read_csv(team_data)
df.head()

,Id,Name,Family Name,Birthdate,Nationality,Category,Start date,End date
0,S000001,Mohammad Zain,Abbas,09/05/1995,Pakistani,SK-N3,19/07/2022,06/09/2022
1,S000002,Kainaat,Amjid,30/12/1996,Pakistani,SK-N1,19/07/2022,13/09/2022
2,S000003,Sergio,Postigo,12/02/1996,Peru,SK-N2,26/07/2022,13/10/2022
3,S000004,Emmanuel,Jamin,28/09/1978,French,SKL-N2,10/01/2015,11/01/2015


In [20]:
column_names = df.columns.to_list()

['Id',
 'Name',
 'Family Name',
 'Birthdate',
 'Nationality',
 'Category',
 'Start date',
 'End date']

In [46]:
team_mapping = join(data_dir, "team_mapping.csv")
map_df = pd.read_csv(team_mapping)
map_df.head()

,column,data_type,type,mapping_code,description
0,Id,string,internal,NaN,Unique identifier for a SEMBU team member
1,Name,external-id,external,P735,First name/given name of SEMBU team member
2,Family Name,external-id,external,P734,Surname/Family name of SEMBU team member
3,Birthdate,external-id,external,P569,Date of birth
4,Nationality,external-id,external,P27,Nationality of SEMBU team member


In [47]:
columns_with_mappings = list(map_df.T.to_dict().values())

In [48]:
columns_with_mappings[0]

{'column': 'Id',
 'data_type': 'string',
 'type': 'internal',
 'mapping_code': nan,
 'description': 'Unique identifier for a SEMBU team member'}

In [49]:
for item in columns_with_mappings:
    __col_name = item['column']
    __data_type = item['data_type']
    __type = item['type']
    __mapping_code = item['mapping_code']
    __description = item['description']

    if __type == 'internal':
        print("Column '{}' is internal.".format(__col_name))

        p = wbi.property.new(datatype=__data_type)
        p.labels.set('en', __col_name)
        p.descriptions.set('en', __description)

        res = p.write()

        ident = [x for x in str(res).split('\n') if "_id='P" in x]
        if len(ident) == 1:
            print(ident[0].split("'")[1])
        else:
            raise Exception("Surprise, this method didn't work.")


Column 'Id' is internal.


ERROR:root:Error while writing to the Wikibase instance
Traceback (most recent call last):
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/entities/baseentity.py", line 255, in _write
    json_result: dict = mediawiki_api_call_helper(data=payload, login=login, allow_anonymous=allow_anonymous, is_bot=is_bot, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 211, in mediawiki_api_call_helper
    return mediawiki_api_call('POST', mediawiki_api_url=mediawiki_api_url, session=session, data=data, headers=headers, max_retries=max_retries, retry_after=retry_after, **kwargs)
  File "/root/anaconda3/envs/wikibase/lib/python3.10/site-packages/wikibaseintegrator/wbi_helpers.py", line 122, in mediawiki_api_call
    raise ModificationFailed(json_data['error'])
wikibaseintegrator.wbi_exceptions.ModificationFailed: 'Property [[Property:P4|P4]] already has label "Id" associated with language code e

ModificationFailed: 'Property [[Property:P4|P4]] already has label "Id" associated with language code en.'